<p style="font-family: Arial; font-size:3em;color:black;"> Project</p>

We will use the titanic dataset from Kaggle (https://www.kaggle.com/). This is a well-known dataset and we will use it for classification- if the passenger survived or passed away.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
training_DF = pd.read_csv('titanic_dataset_GBC.csv')
training_DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
training_DF['Age'].fillna(training_DF['Age'].mean(), inplace=True)

In [4]:
training_DF.drop('Cabin',axis=1,inplace=True)

In [5]:
training_DF.dropna(inplace=True)

In [6]:
sex = pd.get_dummies(training_DF['Sex'])
embark = pd.get_dummies(training_DF['Embarked'])

In [7]:
training_DF.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [8]:
training_DF = pd.concat([training_DF,sex,embark],axis=1)

In [9]:
training_DF.drop(['female','C'],axis=1,inplace=True)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(training_DF.drop(['Survived', 'Age','SibSp','Parch','male','Pclass',],axis=1), 
                                                    training_DF['Survived'], test_size=0.30, 
                                                    random_state=101)

In [12]:
# With the above steps, we have cleaned and pre-processed our data to bed fed to a ML/AI classification model.

# 1. Use Logistic Regression, Decision Tree, Random Forest, SGD, and SVM for classification of the dataset.
# 2. Use GridSearchCV to tune the parameter of each of the above model, can you obtain better results in this step for any of the models? Disucss you observations.
# 3. Randomly (or based on some hypothesis) remove some columns (1,2 or maybe 3 of them) of the dataset features and re-run your classification models. Do you observe any change in classification metrics?
# 4. Submit your project by the midnight of the 8th session. A suggested method is to submit a (text) report to discuss and summarize your findings and an .ipynb with clear comments and code sections to 
#    present your work.

In [13]:
training_DF.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'male', 'Q', 'S'],
      dtype='object')

In [16]:
training_DF.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1
